In [ ]:
import os
from shapely.geometry import Polygon, MultiPolygon
from centerline.geometry import Centerline
import cv2
from skimage.morphology import skeletonize,remove_small_holes, remove_small_objects,medial_axis, binary_erosion, binary_dilation
from skimage.morphology import square, thin
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, \
    subdivide_polygon, label, regionprops
from skimage.segmentation import (morphological_chan_vese,
                                  morphological_geodesic_active_contour,
                                  inverse_gaussian_gradient,
                                  checkerboard_level_set)
from scipy.ndimage import gaussian_filter, binary_fill_holes
import math
from skimage.morphology import convex_hull_image
from skimage.segmentation import active_contour
from skimage.filters import gaussian
from scipy.ndimage import binary_dilation
from scipy.ndimage import distance_transform_edt

import openslide
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
osh  = openslide.OpenSlide("/home/jackson/data/tortuosity_data/_1687394.svs")
img=osh.read_region((0,0),2,osh.level_dimensions[2])
#osh  = openslide.OpenSlide("_1687395.svs")


In [ ]:
basewidth = 300
wpercent = (basewidth/float(img.size[0]))
hsize = int((float(img.size[1])*float(wpercent)))
img = img.resize((basewidth,hsize)) 

In [ ]:
b=np.asarray(img)[:,:,0]<220
b=remove_small_holes(b,1000)
b=remove_small_objects(b,100)

point_set = []
for contour in find_contours(b):
    point_set.extend(contour)



In [ ]:
plt.imshow(b)

In [ ]:
# b=binary_dilation(b,iterations=2)
b = binary_erosion(b)

In [ ]:
plt.imshow(b)

In [ ]:
eb=distance_transform_edt(b)
eb = cv2.normalize(eb, None, 0, 1, cv2.NORM_MINMAX)
eb = (eb)/2

plt.imshow(eb)
plt.colorbar()

In [ ]:
plt.imshow(np.logical_not(b))

In [ ]:
eb2 = distance_transform_edt(np.logical_not(b))
eb2 = cv2.normalize(eb2, None, 0, 1, cv2.NORM_MINMAX)
eb2 = (1-eb2)/2
plt.imshow(eb2**3)
plt.colorbar()

In [ ]:

rp=regionprops(b.astype(np.uint8))[0]
r = np.linspace(rp.bbox[1], rp.bbox[3], 200)
c = np.linspace(rp.bbox[0], rp.bbox[2], 200)
init = np.array([c, r]).T

eb_final = eb2 + eb

plt.imshow(eb_final)
plt.colorbar()


In [ ]:

snake = active_contour(eb_final,init, alpha=0.01, beta=1, gamma=0.001,w_line=100,max_px_move=1,
                       w_edge=0,max_num_iter=2000,convergence=.001,boundary_condition='fixed')
%matplotlib inline
%matplotlib inline
plt.imshow(img)
plt.plot(init[:, 1], init[:, 0], '--g', lw=1)
plt.plot(snake[:, 1], snake[:, 0], lw=1)
plt.show()

In [ ]:
plt.plot(eb[75], label='eb row 75')
plt.plot(eb2[75], label='eb2 row 75')
plt.plot(eb[75] + eb2[75], label='eb + eb2')
plt.legend()

In [ ]:
plt.plot(eb2[75])

In [ ]:
np.linalg.norm(init[-1] - init[0])

In [ ]:
rp.bbox

In [ ]:
plt.imshow(b)
plt.plot(init[:,1],init[:,0])
plt.show()

In [ ]:
plt.imshow(img)

In [ ]:
np.sum(np.linalg.norm(np.diff(snake, axis=0), axis=-1))